## Most Commonly used Optimization Algorithms

| Optimizer | Description | Use Case Example |
|---|---|---|
| SGD | Simple and effective but requires careful tuning of learning rate and momentum | Large-scale training with large datasets |
| Adam | Combines advantages of Adagrad and RMSprop; adaptive learning rates make it suitable for many tasks | Most deep learning tasks, including CNNs and RNNs |
| RMSprop | Adapts the learning rate for each parameter; effective for RNNs and problems where gradients vary significantly | RNNs and problems with varying gradients |

## Importing Requiredd Libraries


In [37]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.src.optimizers.adam import Adam
from keras.optimizers import RMSprop
from keras.optimizers import SGD
import numpy as np
import pandas as pd
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

### Reading Diabetes Dataset

In [38]:
uploaded = files.upload()
filename = next(iter(uploaded))

Saving diabetes.csv to diabetes (1).csv


### Create a DataFrame

In [39]:
diabetes_pd = pd.read_csv(filename)

In [40]:
diabetes_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [41]:
diabetes_pd

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [42]:
diabetes_pd.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [43]:
diabetes_pd.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [44]:
diabetes_pd.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### As this is regression problem, we are separating the independant and dependant variable

In [45]:
X = diabetes_pd.iloc[:, 0:8]
y = diabetes_pd.iloc[:, 8]

In [46]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [47]:
y

,Outcome
0,1
1,0
2,1
3,0
4,1
...,...
763,0
764,0
765,0
766,1


### Sequential Model Building by Adding Layers in NN

In [48]:
model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Compiling the Model

In [49]:
sgd = SGD(learning_rate=0.1,momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

### Train the Model

In [50]:
model.fit(X, y, epochs=100, batch_size=10)

Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6197 - loss: 2.4285
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6523 - loss: 0.6552
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6500 - loss: 0.6514
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6226 - loss: 0.6676
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6659 - loss: 0.6498
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6614 - loss: 0.6435
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6176 - loss: 0.6561
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6555 - loss: 0.6479
Epoch 9/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6647 - loss: 0.6438
Epoch 10/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6441 - loss: 0.6594
Epoch 11/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6676 - loss: 0.6406
Epoch 12/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy:

### Evaluate the Network

In [51]:
loss, accuracy = model.evaluate(X, y)
print(f"Loss: {loss}, Accuracy: {accuracy*100}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6242 - loss: 0.6620  
Loss: 0.64857417345047, Accuracy: 65.10416865348816


### Make Predictions

In [52]:
probabilities = model.predict(X)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [53]:
print(f"\nSGD - Loss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))


SGD - Loss: 0.65, Accuracy: 65.10%


In [68]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 164 (660.00 B)

 Trainable params: 81 (324.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 83 (336.00 B)

### Model Adam

In [54]:
model_adam = Sequential()
model_adam.add(Dense(8, input_dim=8, activation='relu'))
model_adam.add(Dense(1, activation='sigmoid'))

### Compiling the Model

In [55]:
adam = Adam(learning_rate=0.001)
model_adam.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

### Train the Model

In [56]:
model_adam.fit(X, y, epochs=100, batch_size=10)

Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4752 - loss: 10.2476
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4391 - loss: 4.7006
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4736 - loss: 1.7247
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5368 - loss: 1.4497
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5491 - loss: 1.2594
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5276 - loss: 1.3299
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5910 - loss: 0.9892
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5894 - loss: 0.9978
Epoch 9/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6282 - loss: 0.8438
Epoch 10/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6226 - loss: 0.8747
Epoch 11/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5784 - loss: 0.8081
Epoch 12/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy

### Evaluate the Model

In [57]:
loss, accuracy = model_adam.evaluate(X, y)
print(f"Adam - Loss: {loss: .2f}, Accuracy: {accuracy*100: .2f}%")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7186 - loss: 0.5647  
Adam - Loss:  0.54, Accuracy:  73.31%


### Make Predictions

In [58]:
probabilities = model_adam.predict(X)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [59]:
print(f"\nAdam - Loss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))


Adam - Loss: 0.54, Accuracy: 73.31%


In [67]:
model_adam.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 245 (984.00 B)

 Trainable params: 81 (324.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 164 (660.00 B)

### Model RMSPROP

In [60]:
model_rmsprop = Sequential()
model_rmsprop.add(Dense(8, input_dim=8, activation='relu'))
model_rmsprop.add(Dense(1, activation='sigmoid'))

### Compiling the Model

In [61]:
rmsprop = RMSprop(learning_rate=0.001)
model_rmsprop.compile(loss='binary_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

### Train the Model

In [62]:
model_rmsprop.fit(X, y, epochs=100, batch_size=10)

Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6510 - loss: 8.9685
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5577 - loss: 2.5653
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6080 - loss: 1.5668
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6244 - loss: 1.2657
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6672 - loss: 0.9795
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6893 - loss: 0.8478
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6599 - loss: 0.8642
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6444 - loss: 0.8333
Epoch 9/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6634 - loss: 0.7509
Epoch 10/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6836 - loss: 0.6704
Epoch 11/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6324 - loss: 0.7053
Epoch 12/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

### Evaluate the Model

In [63]:
loss, accuracy = model_rmsprop.evaluate(X, y)
print(f"RMSProp - Loss: {loss: .2f}, Accuracy: {accuracy*100: .2f}%")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7227 - loss: 0.5450  
RMSProp - Loss:  0.52, Accuracy:  74.09%


### Make Predictions

In [64]:
probabilities = model_rmsprop.predict(X)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [65]:
print(f"\nRMSProp - Loss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))


RMSProp - Loss: 0.52, Accuracy: 74.09%


In [66]:
model_rmsprop.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 164 (660.00 B)

 Trainable params: 81 (324.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 83 (336.00 B)